## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arcata,US,40.8665,-124.0828,55.92,80,90,0.00,overcast clouds
1,1,Kotka,FI,60.4664,26.9458,51.06,77,99,18.81,overcast clouds
2,2,Provideniya,RU,64.3833,-173.3000,35.74,98,100,7.14,overcast clouds
3,3,Pangnirtung,CA,66.1451,-65.7125,37.40,75,75,3.44,broken clouds
4,4,San Patricio,US,28.0170,-97.5169,88.88,37,0,5.73,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? ')) 


What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_data_df.loc[(city_data_df['Max Temp']<= max_prompt_temp) & (city_data_df['Max Temp']>= min_prompt_temp)]
preferred_temp_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,San Patricio,US,28.0170,-97.5169,88.88,37,0,5.73,clear sky
8,8,Awbari,LY,26.5921,12.7805,82.06,16,0,7.11,clear sky
28,28,Ixtapa,MX,20.7000,-105.2000,86.23,62,1,0.00,clear sky
34,34,Puerto Ayora,EC,-0.7393,-90.3518,80.56,86,77,8.99,broken clouds
37,37,Tame,CO,6.4610,-71.7300,88.84,56,82,1.77,broken clouds
...,...,...,...,...,...,...,...,...,...,...
649,649,Bushehr,IR,28.9684,50.8385,86.09,70,0,6.91,clear sky
651,651,Kampot,KH,10.6167,104.1833,80.65,90,100,6.67,overcast clouds
660,660,Oistins,BB,13.0667,-59.5333,86.09,79,75,21.85,broken clouds
666,666,Thai Binh,VN,20.4500,106.3333,86.00,75,98,9.93,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.count()

City_ID                108
City                   108
Country                108
Lat                    108
Lng                    108
Max Temp               108
Humidity               108
Cloudiness             108
Wind Speed             108
Current Description    108
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_temp_df.dropna()
clean_df.count()

City_ID                108
City                   108
Country                108
Lat                    108
Lng                    108
Max Temp               108
Humidity               108
Cloudiness             108
Wind Speed             108
Current Description    108
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Patricio,US,88.88,clear sky,28.0170,-97.5169,
8,Awbari,LY,82.06,clear sky,26.5921,12.7805,
28,Ixtapa,MX,86.23,clear sky,20.7000,-105.2000,
34,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
37,Tame,CO,88.84,broken clouds,6.4610,-71.7300,
45,Kinshasa,CD,86.38,thunderstorm,-4.3246,15.3215,
47,Butaritari,KI,82.63,scattered clouds,3.0707,172.7902,
62,Hithadhoo,MV,83.21,scattered clouds,-0.6000,73.0833,
83,Saint-Georges,GD,89.28,few clouds,12.0564,-61.7485,
86,Sohag,EG,80.02,clear sky,26.5570,31.6948,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig
